In [2]:
import numpy as np
import pandas as pd
import random
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import ipyvuetify as v

rows_to_keep = 38

sheet_data = pd.read_excel("https://docs.google.com/spreadsheets/d/1DuYUj2ODS8D3PWK42ZopUD1dqcg89ckI6vPn71LidGo/export?format=xlsx")
sheet_data = sheet_data.iloc[:rows_to_keep].drop(columns=["Overall % Lost", "Imgur"]).dropna(axis=1, how="all")
sheet_data = sheet_data.rename(columns={"$": "Pot Contribution", "Starting 1/1/22": "Starting Weight"})
sheet_data = sheet_data.melt(id_vars=["Participant", "Pot Contribution", "Paid?", "Starting Weight"], var_name="Date", value_name="Weight")
sheet_data["Starting Weight"] = pd.to_numeric(sheet_data["Starting Weight"])
sheet_data["Weight"] = pd.to_numeric(sheet_data["Weight"])
sheet_data = sheet_data.dropna(subset=["Weight", "Starting Weight"]).sort_values(by=["Participant", "Date"]).reset_index()

by_participant = sheet_data.groupby("Participant")
by_date = sheet_data.groupby("Date")

sheet_data["Previous Week Weight"] = by_participant["Weight"].shift(1)
sheet_data.loc[sheet_data["Previous Week Weight"].isnull(), "Previous Week Weight"] = sheet_data.loc[sheet_data["Previous Week Weight"].isnull(), "Starting Weight"]
sheet_data["Weight Difference"] = sheet_data["Weight"] - sheet_data["Previous Week Weight"]

sheet_data["Cumulative Weight Lost"] = by_participant["Weight Difference"].transform(pd.Series.cumsum)
sheet_data["Cumulative % Lost"] = sheet_data["Cumulative Weight Lost"] / sheet_data["Starting Weight"] * -100
sheet_data["Participant Index"] = by_participant.ngroup()

participant_colors = ["#%06x" % random.randint(0, 0xFFFFFF) for participant, _ in by_participant]

current_ranks = pd.DataFrame(by_participant["Cumulative % Lost"].max())
current_ranks["rank"] = current_ranks["Cumulative % Lost"].rank(method="first", ascending=False).astype(np.int64)
current_ranks = current_ranks.sort_values(by="rank")


In [3]:
participant_rank_list = []

for index, row in current_ranks.iterrows():
    participant_rank_list.append(v.ListItem(children=[
        v.ListItemIcon(children=[f"{row['rank']:.0f}"]),
        v.ListItemContent(children=[v.ListItemTitle(children=[f"{index}"])]),
        v.ListItemIcon(children=[f"{row['Cumulative % Lost']:.2f}%"]),
    ]))    


total_weight_lost = pd.DataFrame(by_date["Weight Difference"].sum()).reset_index()
total_weight_lost["Total Weight Lost"] = total_weight_lost["Weight Difference"].cumsum()

fig = px.line(total_weight_lost, x="Date", y="Total Weight Lost", title="Total Weight Lost")
fig.update_xaxes(range=[datetime(2022,1,1), datetime(2022, 4, 22)])
fig.update_yaxes(autorange="reversed")
fig.update_layout(barmode='stack')
# fig.show()

total_weight_lost_traces = []

for participant, participant_data in by_participant:
    total_weight_lost_traces.append(go.Bar(name=participant, x=participant_data["Date"], y=participant_data["Cumulative Weight Lost"] * -1))

total_weight_lost_traces.append(go.Scatter(name="Combined Total", x=total_weight_lost["Date"], y=total_weight_lost["Total Weight Lost"] * -1, line=dict(color="black", width=4)))

total_weight_lost_chart = go.FigureWidget(data=total_weight_lost_traces,
                    layout=go.Layout(
                        title=dict(
                            text='Total Weight Lost'
                        ),
                        margin=dict(l=10, r=10, t=50, b=10),
                        barmode='stack'
                    ))

weights_over_time_traces = []
for participant, participant_data in by_participant:
    weights_over_time_traces.append(go.Scatter(name=participant, x=participant_data["Date"], y=participant_data["Weight"]))

avg_weight_over_time = pd.DataFrame(by_date["Weight"].mean()).reset_index()
weights_over_time_traces.append(go.Scatter(name="Average Weight", x=avg_weight_over_time["Date"], y=avg_weight_over_time["Weight"], line=dict(color="black", width=4)))

cumulative_per_lost_traces = []
for participant, participant_data in by_participant:
    cumulative_per_lost_traces.append(go.Scatter(name=participant, x=participant_data["Date"], y=participant_data["Cumulative % Lost"]))

cumulative_per_chart = go.FigureWidget(data=cumulative_per_lost_traces,
                    layout=go.Layout(
                        title=dict(
                            text='Cumulative % Lost'
                        ),
                        margin=dict(l=10, r=10, t=50, b=10),
                    ))


weights_over_time_chart = go.FigureWidget(data=weights_over_time_traces,
                    layout=go.Layout(
                        title=dict(
                            text='Weights over Time'
                        ),
                        margin=dict(l=10, r=10, t=50, b=10),
                        xaxis=dict(range=[datetime(2022,1,1), datetime(2022, 4, 22)])
                    ))

weight_difference_fig = px.bar(sheet_data, x="Date", y="Weight Difference", color='Participant', title="Weight Difference by Week", labels={"Weight Difference":"Weight Difference (lbs)"})
weight_difference_fig.update_yaxes(autorange="reversed")

weight_difference_chart = go.FigureWidget(weight_difference_fig)

v.Container(class_="pa-0", children=[
    v.Row(children=[v.AppBar(color="orange darken-3", dark=True, children=[v.ToolbarTitle(children=["2022 r/LHN Weight Loss Challenge"])])]),
    v.Row(children=[
        v.Col(cols=4, children=[v.Card(outlined=True, children=[
            v.Toolbar(color="orange darken-3", dark=True, children=[v.ToolbarTitle(color="white",children=["Leaderboard"])]),
            v.List(children=participant_rank_list),
        ])]),
        v.Col(cols=8, children=[
            v.Card(outlined=True, children=[total_weight_lost_chart]),            
            v.Card(outlined=True, children=[weight_difference_chart], class_="mt-5"),            
            v.Card(outlined=True, children=[cumulative_per_chart], class_="mt-5"),
            v.Card(outlined=True, children=[weights_over_time_chart], class_="mt-5"),
        ]),
    ])
])

Container(children=[Row(children=[AppBar(children=[ToolbarTitle(children=['2022 r/LHN Weight Loss Challenge'])…